In [47]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from sklearn.utils.class_weight import compute_class_weight

# --- Step 1: Configuration Constants ---
IMAGE_SIZE = (224 , 224)
BATCH_SIZE = 32

def select_dataset_directory():
    """Opens a folder selection dialog and returns the selected path."""
    root = tk.Tk()
    root.withdraw() 
    print("Opening a dialog box to choose your dataset folder...")
    directory_path = filedialog.askdirectory(
        title="Please select your dataset folder (e.g., FINAL DATASET)"
    )
    root.destroy()
    return directory_path


 
def create_data_generators(dataset_dir):
    """Creates and returns training and validation data generators with a 80/20 split."""
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # Reserve 20% of data for validation
    )

    train_generator = datagen.flow_from_directory(
        directory=dataset_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        subset='training',
        color_mode="rgb"   # 👈 force 3-channel input
    )

    validation_generator = datagen.flow_from_directory(
        directory=dataset_dir,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False,
        subset='validation',
        color_mode="rgb"   # 👈 same for validation
    )

    return train_generator, validation_generator

# --- Step 4: Class Weight Calculation ---
def calculate_class_weights(train_generator):
    """Calculates class weights to handle data imbalance."""
    class_labels = np.unique(train_generator.classes)
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=class_labels,
        y=train_generator.classes
    )
    class_weights_dict = dict(zip(class_labels, class_weights))
    return class_weights_dict

tf.keras.backend.clear_session() 
from tensorflow.keras.applications import EfficientNetV2B0

def build_model(num_classes):
    # Use EfficientNetV2B0 as the base
    base_model = EfficientNetV2B0(
        include_top=False,
        weights='imagenet',
        input_shape=IMAGE_SIZE + (3,)  # This will be (224, 224, 3)
    )
    base_model.trainable = False

    # Use the same IMAGE_SIZE here to match the base model
    inputs = tf.keras.Input(shape=IMAGE_SIZE + (3,))  # This will be (224, 224, 3)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)
    return model




In [48]:
# --- Step 6: Model Training Function ---
def train_model(model, train_gen, val_gen, weights):
    """Compiles and trains the model in two phases: head training and fine-tuning."""
    
    # === PHASE 1: TRAIN THE HEAD ===
    print("\n--- PHASE 1: Training the new classification head ---")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    initial_epochs = 10
    history_initial = model.fit(
        train_gen,
        epochs=initial_epochs,
        validation_data=val_gen,
        class_weight=weights  # Apply class weights here!
    )

    # === PHASE 2: FINE-TUNE THE MODEL ===
    print("\n--- PHASE 2: Fine-tuning the top layers of the base model ---")
    # Unfreeze the base model.
    model.layers[1].trainable = True
    
    # We'll only unfreeze the top 30 layers.
    for layer in model.layers[1].layers[:-30]:
        layer.trainable = False

    # Re-compile the model with a very low learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    fine_tune_epochs = 10
    total_epochs = initial_epochs + fine_tune_epochs
    
    history_fine_tune = model.fit(
        train_gen,
        epochs=total_epochs,
        initial_epoch=history_initial.epoch[-1],
        validation_data=val_gen,
        class_weight=weights
    )

    return history_initial, history_fine_tune

# --- Main Execution Block ---
if __name__ == "__main__":
    DATASET_DIR = select_dataset_directory()

    if DATASET_DIR:
        print(f"\n✅ Dataset folder selected: {DATASET_DIR}")

        # Steps 3 & 4
        train_gen, val_gen = create_data_generators(DATASET_DIR)
        num_classes = train_gen.num_classes
        weights = calculate_class_weights(train_gen)
        
        # Step 5
        model = build_model(num_classes=num_classes)
        
        # Step 6: Call the training function
        history_initial, history_fine_tune = train_model(model, train_gen, val_gen, weights)

        print("\n🎉 Training complete! 🎉")

    else:
        print("\n❌ No folder was selected.")

Opening a dialog box to choose your dataset folder...

✅ Dataset folder selected: C:/Users/Jay Surieya/Downloads/FINAL DATASET
Found 47928 images belonging to 110 classes.
Found 11975 images belonging to 110 classes.

--- PHASE 1: Training the new classification head ---


c:\Users\Jay Surieya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1703s 1s/step - accuracy: 0.5101 - loss: 2.3728 - val_accuracy: 0.6367 - val_loss: 1.4648
Epoch 2/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1599s 1s/step - accuracy: 0.6158 - loss: 1.4565 - val_accuracy: 0.6704 - val_loss: 1.2885
Epoch 3/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 928s 619ms/step - accuracy: 0.6406 - loss: 1.2768 - val_accuracy: 0.6788 - val_loss: 1.2253
Epoch 4/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1047s 662ms/step - accuracy: 0.6534 - loss: 1.1513 - val_accuracy: 0.6872 - val_loss: 1.1885
Epoch 5/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1851s 1s/step - accuracy: 0.6621 - loss: 1.0995 - val_accuracy: 0.6928 - val_loss: 1.1709
Epoch 6/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1170s 781ms/step - accuracy: 0.6697 - loss: 1.0442 - val_accuracy: 0.6914 - val_loss: 1.1438
Epoch 7/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1067s 712ms/step - accuracy: 0.6751 - loss: 0.9850 - val_accuracy: 0.7004 - val_loss: 1.1219
Epoch 8/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 1034s 690ms/step - accu

In [49]:
model.save("food_classifier_final_model.keras")

print("\n✅ Model saved successfully!")
print("The file 'food_classifier_final_model.keras' has been created in your project directory.")


✅ Model saved successfully!
The file 'food_classifier_final_model.keras' has been created in your project directory.


In [50]:
import tensorflow as tf

# The path to your saved model file
MODEL_PATH = "food_classifier_final_model.keras"

print(f"Attempting to load model from: {MODEL_PATH}")

try:
    # Load the model from the file
    loaded_model = tf.keras.models.load_model(MODEL_PATH)
    
    # If loading is successful, print the model's summary
    print("\n✅ Model loaded successfully! The file is not corrupted.")
    print("Here is the model summary:")
    loaded_model.summary()
    
except Exception as e:
    # If an error occurs, the file is likely corrupted or saved incorrectly
    print(f"\n❌ Error loading the model: {e}")

Attempting to load model from: food_classifier_final_model.keras

✅ Model loaded successfully! The file is not corrupted.
Here is the model summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 110)            │       140,910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,669,788 (33.07 MB)

 Trainable params: 1,304,782 (4.98 MB)

 Non-trainable params: 4,755,440 (18.14 MB)

 Optimizer params: 2,609,566 (9.95 MB)